In [ ]:
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
from patsy import bs

# Cargar los datos
ozono = pd.read_csv("anualmean_ozono.csv")
temperatura = pd.read_csv("anualmean_temperatura.csv")
mortalidad = pd.read_excel("data_mortalidad.xlsx")

# Convertir los nombres de localidad a mayúsculas en todos los DataFrames
mortalidad["NOMBRE_LOCALIDAD"] = mortalidad["NOMBRE_LOCALIDAD"].str.upper()
ozono["NOMBRE_LOCALIDAD"] = ozono["NOMBRE_LOCALIDAD"].str.upper()
temperatura["NOMBRE_LOCALIDAD"] = temperatura["NOMBRE_LOCALIDAD"].str.upper()

# Diccionario para reemplazar nombres en mortalidad
mapeo_nombres = {
    "ANTONIO NARIÑO": None,  # No está en ozono ni temperatura
    "BARRIOS UNIDOS": None,  # No está en ozono ni temperatura
    "BOSA": "BOSA",
    "CHAPINERO": "CHAPINERO",
    "CIUDAD BOLÍVAR": "CIUDAD BOLIVAR",
    "ENGATIVÁ": "ENGATIVA",
    "FONTIBÓN": "FONTIBON",
    "KENNEDY": "KENNEDY",
    "LA CANDELARIA": None,  # No está en ozono ni temperatura
    "LOS MÁRTIRES": None,  # No está en ozono ni temperatura
    "PUENTE ARANDA": "PUENTE ARANDA",
    "RAFAEL URIBE URIBE": None,  # No está en ozono ni temperatura
    "SAN CRISTÓBAL": "SAN CRISTOBAL",
    "SANTA FE": "SANTA FE",
    "SUBA": "SUBA",
    "SUMAPAZ": None,  # No está en ozono ni temperatura
    "TEUSAQUILLO": "TEUSAQUILLO",
    "TUNJUELITO": "TUNJUELITO",
    "USAQUÉN": "USAQUEN",
    "USME": "USME"
}

# Aplicar la transformación en `mortalidad`
mortalidad["NOMBRE_LOCALIDAD"] = mortalidad["NOMBRE_LOCALIDAD"].map(mapeo_nombres)

# Eliminar las filas con localidades que no tienen datos en ozono y temperatura
mortalidad = mortalidad.dropna(subset=["NOMBRE_LOCALIDAD"])

# Llenar valores NaN en Temperatura con el promedio de la misma NOMBRE_LOCALIDAD
temperatura["Temperatura"] = temperatura.groupby("NOMBRE_LOCALIDAD")["Temperatura"].transform(lambda x: x.fillna(x.mean()))
ozono["Ozono"] = ozono.groupby("NOMBRE_LOCALIDAD")["Ozono"].transform(lambda x: x.fillna(x.mean()))

# Filtrar datos de mortalidad desde 2013
mortalidad = mortalidad[mortalidad["AÑO"] >= 2013].reset_index(drop=True)

# Ajustar Ozono con la constante beta
ozono["Ozono_Ajustado"] = 1372.1 * ozono["Ozono"]

# Unir los datos solo cuando se necesiten para el modelo
data = mortalidad.merge(ozono, on=["NOMBRE_LOCALIDAD", "AÑO"], how="left")
data = data.merge(temperatura, on=["NOMBRE_LOCALIDAD", "AÑO"], how="left")
data = data.dropna().reset_index(drop=True)

# Verificar si hay datos suficientes después del cruce
if data.empty:
    raise ValueError("No hay datos suficientes después de la fusión.")

# Definir la fórmula del modelo
formula = 'Muertes ~ Ozono_Ajustado + bs(Temperatura, df=6)'

# Ajustar el modelo de Poisson con sobre-dispersión
model = smf.glm(formula, data=data, family=sm.families.Poisson()).fit()

# Mostrar resultados
print(model.summary())


                 Generalized Linear Model Regression Results                  
Dep. Variable:                Muertes   No. Observations:                  108
Model:                            GLM   Df Residuals:                      100
Model Family:                 Poisson   Df Model:                            7
Link Function:                    Log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:            -6.0215e+05
Date:                Tue, 25 Mar 2025   Deviance:                   1.2031e+06
Time:                        13:18:50   Pearson chi2:                 1.39e+06
No. Iterations:                     5   Pseudo R-squ. (CS):              1.000
Covariance Type:            nonrobust                                         
                               coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------
Intercept               

In [ ]:
data

,NOMBRE_LOCALIDAD,AÑO,Muertes,Ozono,Ozono_Ajustado,Temperatura
0,BOSA,2013,12873,3.115973,4275.426308,14.390687
1,BOSA,2014,13433,2.907036,3988.744052,14.390687
2,BOSA,2015,13608,2.698099,3702.061796,14.390687
3,BOSA,2016,15393,2.488661,3414.692288,14.390687
4,BOSA,2017,14896,2.279653,3127.911853,14.390687
...,...,...,...,...,...,...
103,USME,2017,6190,12.018157,16490.113734,14.698660
104,USME,2018,6715,12.018157,16490.113734,14.698660
105,USME,2019,6620,12.018157,16490.113734,14.698660
106,USME,2020,9455,12.578691,17259.221934,14.792046


In [ ]:

# Agregar predicciones al DataFrame
data["Muertes_Predichas"] = model.predict(data)

# Calcular % de diferencia entre predicción y muertes reales
#data["% Mortalidad"] = ((data["Muertes_Predichas"] - data["Muertes"]) / data["Muertes"]) * 100
data["% Mortalidad"] = (data["Muertes_Predichas"] / data["Muertes"]) * 100 - 100

tabla_mortalidad = data[["NOMBRE_LOCALIDAD", "AÑO", "% Mortalidad"]]
tabla_mortalidad = data.groupby(["NOMBRE_LOCALIDAD", "AÑO"])["% Mortalidad"].mean().reset_index()
print(tabla_mortalidad)


    NOMBRE_LOCALIDAD   AÑO  % Mortalidad
0               BOSA  2013    130.354274
1               BOSA  2014    120.796125
2               BOSA  2015    118.001048
3               BOSA  2016     92.760588
4               BOSA  2017     99.232552
..               ...   ...           ...
103             USME  2017    163.365715
104             USME  2018    142.774948
105             USME  2019    146.258879
106             USME  2020     60.389643
107             USME  2021     45.183009

[108 rows x 3 columns]


In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
data[["Ozono", "Temperatura"]] = scaler.fit_transform(data[["Ozono", "Temperatura"]])


In [ ]:
import pandas as pd
import numpy as np
from patsy import dmatrix
import statsmodels.api as sm
import matplotlib.pyplot as plt

# Sample dataframe structure
np.random.seed(42)
n = 365 * 3  # e.g. 3 years of daily data

df = pd.DataFrame({
    'day': pd.date_range(start='2010-01-01', periods=n, freq='D'),
    'DOW': pd.date_range(start='2010-01-01', periods=n, freq='D').dayofweek,
    'O3': np.random.normal(50, 10, size=n),  # O3 concentrations
    'Temp': np.random.normal(20, 5, size=n),  # temperature
    'Dew': np.random.normal(10, 3, size=n),  # dew point
    'AgeGroup': np.random.choice(['<65', '65-74', '75+'], size=n),
    'Deaths': np.random.poisson(5, size=n)  # observed deaths
})

# Add lagged variables
df['Temp_lag'] = df['Temp'].rolling(window=3).mean().shift(1)
df['Dew_lag'] = df['Dew'].rolling(window=3).mean().shift(1)

# Time variable in years since start
df['time'] = (df['day'] - df['day'].min()).dt.days / 365

# Create spline terms
spline_time = dmatrix("bs(time, df=21, degree=3)", {"time": df['time']}, return_type='dataframe')
spline_temp = dmatrix("bs(Temp, df=6, degree=3)", {"Temp": df['Temp']}, return_type='dataframe')
spline_temp_lag = dmatrix("bs(Temp_lag, df=6, degree=3)", {"Temp_lag": df['Temp_lag']}, return_type='dataframe')
spline_dew = dmatrix("bs(Dew, df=3, degree=3)", {"Dew": df['Dew']}, return_type='dataframe')
spline_dew_lag = dmatrix("bs(Dew_lag, df=3, degree=3)", {"Dew_lag": df['Dew_lag']}, return_type='dataframe')

# Create design matrix
X = pd.concat([
    df[['O3']],  # ozone
    pd.get_dummies(df['DOW'], prefix='DOW', drop_first=True),  # day of week
    spline_time,
    spline_temp,
    spline_temp_lag,
    spline_dew,
    spline_dew_lag,
    pd.get_dummies(df['AgeGroup'], prefix='Age', drop_first=True)
], axis=1).dropna()

# Target variable (trimmed to match X after dropping NA rows)
y = df.loc[X.index, 'Deaths']

# Fit an over-dispersed Poisson model (quasi-Poisson)
model = sm.GLM(y, X, family=sm.families.Poisson()).fit()

# Display summary
print(model.summary())

In [ ]:
from google.colab import files
import pandas as pd

uploaded = files.upload()
df = pd.read_csv(next(iter(uploaded)))

# Preprocess and generate variables
import numpy as np
from patsy import dmatrix
import statsmodels.api as sm
import matplotlib.pyplot as plt

# Convert day to datetime and compute additional features
df['day'] = pd.to_datetime(df['day'])
df['DOW'] = df['day'].dt.dayofweek
df['Temp_lag'] = df['Temp'].rolling(window=3).mean().shift(1)
df['Dew_lag'] = df['Dew'].rolling(window=3).mean().shift(1)
df['time'] = (df['day'] - df['day'].min()).dt.days / 365

# Drop rows with missing values (due to rolling lags)
df = df.dropna()

# Create spline terms
spline_time = dmatrix("bs(time, df=21, degree=3)", {"time": df['time']}, return_type='dataframe')
spline_temp = dmatrix("bs(Temp, df=6, degree=3)", {"Temp": df['Temp']}, return_type='dataframe')
spline_temp_lag = dmatrix("bs(Temp_lag, df=6, degree=3)", {"Temp_lag": df['Temp_lag']}, return_type='dataframe')
spline_dew = dmatrix("bs(Dew, df=3, degree=3)", {"Dew": df['Dew']}, return_type='dataframe')
spline_dew_lag = dmatrix("bs(Dew_lag, df=3, degree=3)", {"Dew_lag": df['Dew_lag']}, return_type='dataframe')

# Build design matrix
X = pd.concat([
    df[['O3']],
    pd.get_dummies(df['DOW'], prefix='DOW', drop_first=True),
    spline_time,
    spline_temp,
    spline_temp_lag,
    spline_dew,
    spline_dew_lag,
    pd.get_dummies(df['AgeGroup'], prefix='Age', drop_first=True)
], axis=1)

y = df['Deaths']

#Fit Poisson model
model = sm.GLM(y, X, family=sm.families.Poisson()).fit()
df['mu_hat'] = model.predict(X)

#Plot observed vs predicted
plt.figure(figsize=(16, 6))
plt.plot(df['day'], df['Deaths'], label='Observed Deaths', color='orange', alpha=0.4, linewidth=1.5)
plt.plot(df['day'], df['mu_hat'], label='Predicted Deaths (μ̂)', color='firebrick', linewidth=2)
plt.title("Observed vs Predicted Deaths Over Time", fontsize=18, fontweight='bold')
plt.xlabel("Date", fontsize=14)
plt.ylabel("Number of Deaths", fontsize=14)
plt.xticks(rotation=45)
plt.grid(True, linestyle='--', alpha=0.6)
plt.legend(fontsize=12, loc='upper right')
plt.tight_layout()
plt.show()

In [ ]:
import pandas as pd
import numpy as np
from patsy import dmatrix

# Leer archivos
ozone = pd.read_csv('llenado_O3.csv', parse_dates=['FECHA'])
temp = pd.read_csv('llenado_TEM.csv', parse_dates=['FECHA'])
dew = pd.read_csv('llenado_HR.csv', parse_dates=['FECHA'])

# Renombrar columnas
ozone.rename(columns={'MEAN': 'O3_MEAN'}, inplace=True)
temp.rename(columns={'MEAN': 'TEMP_MEAN'}, inplace=True)
dew.rename(columns={'MEAN': 'DEW_MEAN'}, inplace=True)

# Unir por fecha
df = pd.merge(ozone, temp, on='FECHA')
df = pd.merge(df, dew, on='FECHA')
df = df.sort_values('FECHA').reset_index(drop=True)

# Variables temporales
df['DOW'] = df['FECHA'].dt.dayofweek
df['time_t'] = (df['FECHA'] - df['FECHA'].min()).dt.days

# Promedios móviles
df['O3_avg_0_3'] = df['O3_MEAN'].rolling(window=4, min_periods=1).mean()
df['T_t'] = df['TEMP_MEAN']
df['T_t_1_3'] = df['TEMP_MEAN'].shift(1).rolling(window=3, min_periods=1).mean()
df['D_t'] = df['DEW_MEAN']
df['D_t_1_3'] = df['DEW_MEAN'].shift(1).rolling(window=3, min_periods=1).mean()

# Eliminar filas con NaN generadas por los shift
df = df.dropna().reset_index(drop=True)

# Crear splines naturales
spl_time = dmatrix("bs(time_t, df=7, include_intercept=False)", data=df, return_type='dataframe')
spl_Tt = dmatrix("bs(T_t, df=6, include_intercept=False)", data=df, return_type='dataframe')
spl_Tt_1_3 = dmatrix("bs(T_t_1_3, df=6, include_intercept=False)", data=df, return_type='dataframe')
spl_Dt = dmatrix("bs(D_t, df=3, include_intercept=False)", data=df, return_type='dataframe')
spl_Dt_1_3 = dmatrix("bs(D_t_1_3, df=3, include_intercept=False)", data=df, return_type='dataframe')

# Renombrar columnas spline para claridad
spl_time.columns = [f'spline_time_{i}' for i in range(spl_time.shape[1])]
spl_Tt.columns = [f'spline_Tt_{i}' for i in range(spl_Tt.shape[1])]
spl_Tt_1_3.columns = [f'spline_Tt_1_3_{i}' for i in range(spl_Tt_1_3.shape[1])]
spl_Dt.columns = [f'spline_Dt_{i}' for i in range(spl_Dt.shape[1])]
spl_Dt_1_3.columns = [f'spline_Dt_1_3_{i}' for i in range(spl_Dt_1_3.shape[1])]

# Unir todo al DataFrame principal
df_final = pd.concat([df, spl_time, spl_Tt, spl_Tt_1_3, spl_Dt, spl_Dt_1_3], axis=1)

# Mostrar columnas finales
print(df_final.columns)
print(df_final.head())


Index(['FECHA', 'O3_MEAN', 'TEMP_MEAN', 'DEW_MEAN', 'DOW', 'time_t',
       'O3_avg_0_3', 'T_t', 'T_t_1_3', 'D_t', 'D_t_1_3', 'spline_time_0',
       'spline_time_1', 'spline_time_2', 'spline_time_3', 'spline_time_4',
       'spline_time_5', 'spline_time_6', 'spline_time_7', 'spline_Tt_0',
       'spline_Tt_1', 'spline_Tt_2', 'spline_Tt_3', 'spline_Tt_4',
       'spline_Tt_5', 'spline_Tt_6', 'spline_Tt_1_3_0', 'spline_Tt_1_3_1',
       'spline_Tt_1_3_2', 'spline_Tt_1_3_3', 'spline_Tt_1_3_4',
       'spline_Tt_1_3_5', 'spline_Tt_1_3_6', 'spline_Dt_0', 'spline_Dt_1',
       'spline_Dt_2', 'spline_Dt_3', 'spline_Dt_1_3_0', 'spline_Dt_1_3_1',
       'spline_Dt_1_3_2', 'spline_Dt_1_3_3'],
      dtype='object')
                FECHA   O3_MEAN  TEMP_MEAN   DEW_MEAN  DOW  time_t  \
0 2013-01-01 01:00:00  7.479532  10.622880  76.238210    1       0   
1 2013-01-01 02:00:00  7.393653  10.279821  76.757645    1       0   
2 2013-01-01 03:00:00  5.894715   9.837080  77.975899    1       0   
3 201

In [ ]:
X = df_final[[
    'O3_avg_0_3', 'DOW',
    # Splines de tiempo
    'spline_time_0', 'spline_time_1', 'spline_time_2', 'spline_time_3',
    'spline_time_4', 'spline_time_5', 'spline_time_6', 'spline_time_7',
    # Splines de temperatura actual
    'spline_Tt_0', 'spline_Tt_1', 'spline_Tt_2', 'spline_Tt_3',
    'spline_Tt_4', 'spline_Tt_5', 'spline_Tt_6',
    # Splines de temperatura rezagada
    'spline_Tt_1_3_0', 'spline_Tt_1_3_1', 'spline_Tt_1_3_2',
    'spline_Tt_1_3_3', 'spline_Tt_1_3_4', 'spline_Tt_1_3_5', 'spline_Tt_1_3_6',
    # Splines de dew point
    'spline_Dt_0', 'spline_Dt_1', 'spline_Dt_2', 'spline_Dt_3',
    # Splines de dew point rezagado
    'spline_Dt_1_3_0', 'spline_Dt_1_3_1', 'spline_Dt_1_3_2', 'spline_Dt_1_3_3'
]]


In [ ]:
X = sm.add_constant(X)
poisson_model = sm.GLM(y, X, family=sm.families.Poisson())
poisson_results = poisson_model.fit(cov_type='HC0')
print(poisson_results.summary())

In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from patsy import dmatrices

# Simular mortalidad si no tienes aún
np.random.seed(42)
df_final['deaths'] = np.random.poisson(lam=10, size=len(df_final))  # Reemplaza con tus datos reales

# Escalar O3 a cada 10 ppb
df_final['O3_10ppb'] = df_final['O3_avg_0_3'] / 10

# Fórmula del modelo con splines y día de la semana
spline_cols = [col for col in df_final.columns if 'spline' in col]
spline_formula = ' + '.join(spline_cols)

formula = f"""deaths ~ O3_10ppb + C(DOW) + {spline_formula}"""

# Crear diseño de matrices
y, X = dmatrices(formula, data=df_final, return_type='dataframe')

# Ajustar modelo de Poisson con overdispersion (quasi-Poisson ≈ modelo de Poisson con errores robustos)
poisson_model = sm.GLM(y, X, family=sm.families.Poisson())
poisson_results = poisson_model.fit(cov_type='HC0')  # Robust standard errors

# Mostrar resumen
print(poisson_results.summary())


LinAlgError: Singular matrix

In [ ]:
duplicadas = X.T.duplicated()
print(X.columns[duplicadas])

Index(['spline_time_0', 'spline_Tt_0', 'spline_Tt_1_3_0', 'spline_Dt_0',
       'spline_Dt_1_3_0'],
      dtype='object')
